In [1]:
import polars as pl
import os 
from joblib import load

df = pl.read_csv("../Data/Titanic/test.csv")

In [2]:
def standardize(column):
    return (column - column.mean()) / column.std()

df = df.with_columns(
    pl.col("Age").fill_null(pl.col("Age").median()),
    pl.col("Fare").fill_null(pl.col("Fare").median())
    ).with_columns(
        pl.col("Fare").log1p().alias("Fare"),
        ).to_dummies([
            "Embarked",
            "Sex"
        ]).drop([
            "Name",
            "Ticket",
            "Cabin",
            "Embarked_S",
            "Embarked_null",
            "Sex_male"
            ],
            strict=False
            ).with_columns(
                standardize(pl.col("Age")).alias("Age"),
                standardize(pl.col("Fare")).alias("Fare")
            )

df.head()

PassengerId,Pclass,Sex_female,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q
i64,i64,u8,f64,i64,i64,f64,u8,u8
892,3,0,0.385769,0,0,-0.865993,0,1
893,3,1,1.369729,1,0,-0.967989,0,0
894,2,0,2.550481,0,0,-0.668451,0,1
895,3,0,-0.204607,0,0,-0.772721,0,0
896,3,1,-0.598191,1,1,-0.443255,0,0


In [3]:
os.listdir("../Models/Titanic")

['GradientBoostingClassifier__Pclass__Sex_female__Age__SibSp__Parch__Fare__Embarked_C__Embarked_Q.joblib',
 'GradientBoostingClassifier__Sex_female__Embarked_C__IsChild__HasSmallFamily__PC1__PC2__PC3.joblib',
 'GradientBoostingClassifier__Sex_female__Embarked_C__IsChild__HasSmallFamily__PC1__PC3.joblib']

In [4]:
files = [file for file in os.listdir("../Models/Titanic")]
model = load(f'../Models/Titanic/{files[0]}')

X = df.drop("PassengerId")
index = df["PassengerId"]
predictions = model.predict(X)

solution = pl.DataFrame({
    "PassengerId" : index,
    "Survived" : predictions
    })

solution.write_csv("../Data/Titanic/solution.csv")